## TO DO
add accuracy plots, confusion matrix

kfolds, stratified kfold

scores by # words used for classification and dataset

track best words for classification

PCA?

In [487]:
#from imblearn.over_sampling import RandomOverSampler, SMOTE
#import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA

import numpy as np
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn import svm 
from sklearn.metrics import accuracy_score
from statistics import mean
import random
from scipy.io import loadmat

In [488]:
# Variables to tweak
num_words = 20
dset = 'dl'
#kfolds = 

In [489]:
if dset = 'dl':
    x = loadmat('dataset_lab_276_dl.mat')
    data = x['csid_lab']
    print(x['csid_lab'][0].shape)
elif dset = 'ul':
    x = loadmat('dataset_lab_276_ul.mat')
    data = x['csiu_lab']
    print(x['csiu_lab'][0].shape)
elif dset = 'home':
    x = loadmat('dataset_home_276.mat')
    data = x['csid_home']
    print(x['csid_home'][0].shape)
    
print(x.keys())

['csid_lab', '__version__', 'label_lab', '__header__', '__globals__']


In [468]:
with open('sign_labels.csv') as f:
    labels = f.readlines()[0].split('\r')
    labels[0] = labels[0][3:]               # whitespace error with first element
    print(len(labels))

# make ground truth for each element
label_ind = [i[0]-1 for i in x['label_lab']]
print(len(label_ind))
print(labels[label_ind[0]]) #first label in dataset

276
5520
FINISH


In [490]:
random_words = []
while(len(random_words)!=num_words):
    a = int(random.random() * 276)
    if a not in random_words:
        random_words.append(a)

In [479]:
#
#
label_indices = []

for i in random_words:
    label_indices.append(np.where(x['label_lab']==[i])[0]) # indices of where labels are the same 
                                                     # throughout dataset (same words)

# words being used
for i in range(0, len(label_indices)):
    print(labels[label_indices[i][0]])


(array([  16,  292,  568,  844, 1120, 1396, 1672, 1948, 2224, 2500, 2776,
       3052, 3328, 3604, 3880, 4156, 4432, 4708, 4984, 5260]), 0)
ANIMAL
(array([  15,  291,  567,  843, 1119, 1395, 1671, 1947, 2223, 2499, 2775,
       3051, 3327, 3603, 3879, 4155, 4431, 4707, 4983, 5259]), 1)
YES
(array([ 274,  550,  826, 1102, 1378, 1654, 1930, 2206, 2482, 2758, 3034,
       3310, 3586, 3862, 4138, 4414, 4690, 4966, 5242, 5518]), 2)
CLOSE
(array([ 202,  478,  754, 1030, 1306, 1582, 1858, 2134, 2410, 2686, 2962,
       3238, 3514, 3790, 4066, 4342, 4618, 4894, 5170, 5446]), 3)
DRAW
(array([  79,  355,  631,  907, 1183, 1459, 1735, 2011, 2287, 2563, 2839,
       3115, 3391, 3667, 3943, 4219, 4495, 4771, 5047, 5323]), 4)
CHILDREN
(array([ 171,  447,  723,  999, 1275, 1551, 1827, 2103, 2379, 2655, 2931,
       3207, 3483, 3759, 4035, 4311, 4587, 4863, 5139, 5415]), 5)
KITCHEN
(array([ 269,  545,  821, 1097, 1373, 1649, 1925, 2201, 2477, 2753, 3029,
       3305, 3581, 3857, 4133, 4409, 4685, 4961

In [480]:
# randomly sample percent% from each sample for testing

def get_rand_indices(size, percent):
    test_set_size = int(size * percent)
    test_set_ind = []

    while(len(test_set_ind)<=test_set_size):
        rand = int(random.random() * size)
        if rand not in test_set_ind:
            test_set_ind.append(rand)
    return test_set_ind


get_rand_indices(20, .2)

[13, 18, 9, 4, 1]

In [481]:
x_train = []
x_test = []
y_train = []
y_test = []

for i in label_indices:
    test_indices = get_rand_indices(len(i), .2)
    train_indices = [j for j in range(0, len(i)) if j not in test_indices]
    for j in test_indices:
        x_test.append(data[:,:,:,i[j]])
        y_test.append(i[j]%276)
        
    for j in train_indices:
        x_train.append(data[:,:,:,i[j]])
        y_train.append(i[j]%276)

In [482]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

(300, 200, 30, 3)
(300,)
(100, 200, 30, 3)
(100,)
20


In [483]:
# simplify indexes to 0 - num_words

def to_categorical_(train, test):
    cur = 0
    words = dict.fromkeys(test)
    for i in words.keys():
        words[i] = cur
        cur += 1
    return ([words[i] for i in train], [words[i] for i in test])

y_train, y_test = to_categorical_(y_train, y_test)

In [484]:
print(y_test)

[9, 9, 9, 9, 9, 18, 18, 18, 18, 18, 11, 11, 11, 11, 11, 5, 5, 5, 5, 5, 8, 8, 8, 8, 8, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 3, 3, 3, 3, 3, 13, 13, 13, 13, 13, 10, 10, 10, 10, 10, 4, 4, 4, 4, 4, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 14, 14, 14, 14, 14, 2, 2, 2, 2, 2, 19, 19, 19, 19, 19, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 12, 12, 12, 12, 12, 15, 15, 15, 15, 15]


In [485]:
x_test = map(abs, x_test)
x_train = map(abs, x_train)

test_samples, testx, testy, testz = np.shape(x_test)
train_samples, trainx, trainy, trainz = np.shape(x_train)

x_test = np.reshape(x_test, (test_samples, testx*testy*testz))
x_train = np.reshape(x_train, (train_samples, trainx*trainy*trainz))

print(np.shape(x_test))
print(np.shape(x_train))

(100, 18000)
(300, 18000)


In [486]:
y_allPred = []
y_allVal = []

clf = svm.SVC(gamma='auto', kernel='rbf', decision_function_shape='ovo')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
y_allPred = np.append(y_allPred, y_pred)
y_allVal = np.append(y_allVal, y_test)
print (accuracy_score(y_test, y_pred))

0.75
